In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Ziwen 


In [ ]:
# Packages for data processing
import re
import pandas as pd
import string
import spacy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#!pip install wordcloud
from wordcloud import WordCloud

#Package for feature extraction
import nltk.data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize   
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer



#Packages for building a classifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from statistics import mode

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Stops words for pre-processing
nltk.download('stopwords')
stop_word = stopwords.words('english')

#Initialising the word-tokenizer
word_tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?")

# Defining the Stemmer and Lemmatizer   
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4)/gender_classification/train_labels.csv") # reading labeled data
# hashtags = ['#VoteThemOut',
#             # '#ToryScumOut',
#             'F1',
#             'Lisa',
#             '#EnoughIsEnough',
#             '#EnergyPrices',
#             '#iOS16',
#             '#taiwan',
#             '#onepiece',
#             '#CostOfLivingCrisis',
#             '#GetBackToWorkYouFatPonce', 
#             # '#ClosingCeremony',
#             '#BookLoversDay',
#             '#biden']
# for hash in hashtags:
#         # device = torch.device("cuda")

#         test_df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/ran_untitled/{hash}_USER_scores_with_offensiveness_irony.csv')
# test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/gender_classification/test_labels.csv") # reading test data

In [ ]:
train_df.iloc[0]

id        d7d392835f50664fc079f0f388e147a0
gender                                male
Name: 0, dtype: object

In [ ]:
#XML file is read and displayed
with open('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/data1/d7d392835f50664fc079f0f388e147a0.xml','r',encoding='cp437') as in_file:
    xml_first_data = in_file.read()
    
# xml_data is a string and the first 1000 chracters are printed, ready for inspection
print(xml_first_data[0:1000])



<author lang="en">
	<documents>
		<document><![CDATA[@CSIFERROSCAN youch! Good things to know! Is that sort of stuff repairable?]]></document>
		<document><![CDATA[Succumbed to FOMO and bought GNR tickets. Remember asking parents to buy GNR Lies album as a young boy because it had boobies inside. ≡ƒÿé]]></document>
		<document><![CDATA[Brown eye Broom a cool 73 then to the rescue to clear out some broken window in Wellington @TheACCnz https://t.co/eoYmdswYX5]]></document>
		<document><![CDATA[Shout out to Auckland tennis fans who get to sleep in this morning but worth it to watch the ≡ƒÉÉ win #18 #Federer #needcoffee]]></document>
		<document><![CDATA[@nickyoung210 Someone had some balls to come up with that ≡ƒÿé]]></document>
		<document><![CDATA[@okcthunder Hey guys, are there any more releases coming of the 2016 Christmas day jerseys?]]></document>
		<document><![CDATA[@JulieReddish Yeah but curiosity got the better of me!! ≡ƒæÇ]]></document>
		<document><![CDATA[@JulieReddish I rew

In [ ]:
#Display the size of the train and test dataset
print("Length of training dataset is:",len(train_df))
# print("Length of testing dataset is:",len(test_df))

#Dimensions of the train and test dataset
print("Shape of the train dataset",train_df.shape)
# print("Shape of the test dataset",test_df.shape)

Length of training dataset is: 3100
Shape of the train dataset (3100, 2)


In [ ]:
## Looking at the labels ratio - train set
print('No of tweets by Male (out of 3100) - ',train_df.gender[train_df.gender=='male'].count())
print('No of tweets by Female (out of 3100) - ',train_df.gender[train_df.gender=='female'].count())

No of tweets by Male (out of 3100) -  1552
No of tweets by Female (out of 3100) -  1548


In [ ]:
#Function to preprocess the data and read the XML files
def preprocess(xml_data):
    pattern_text = pattern_text = r'<!\[CDATA\[(.*?)\]\]>' #extracting the sentences
    # Text extraction
    xml_text = re.findall(pattern_text, xml_data,  flags=re.DOTALL)
    #for every text file unwanted characters are being removed
    for i in range(0,len(xml_text)):
       

        # Removal of unwanted characters
        xml_text[i] = re.sub('@[^\s]* ', '',  xml_text[i]) #removing what is starting from @
        xml_text[i] = re.sub(' https://.*', '', xml_text[i]) #removing all the https tags
        xml_text[i] = re.sub('[^A-Za-z0-9\s]*', '', xml_text[i]) #removing words and special characters which are not alphabets

        # Case Normalisation
        xml_text[i] = xml_text[i].lower()
    
    xml_text = ' '.join(xml_text)
    return(xml_text)

In [ ]:
# Reading XML files and Extracting the XML content
# Defining a function to read XML files and append the preprocessed sentences as done above

#Creating the empty lists 
train_vec = []
pre_processed_data=[]
#function to read the XML files which must be preprocessed
def read_xml(file_name):
    with open(file_name,'r',encoding='cp437') as in_file: #read the xml data
        xml_data = in_file.read()
        
        # Calling function to pre-process data
        pre_processed_dt = preprocess(xml_data)
        pre_processed_data.append(pre_processed_dt)
        return(pre_processed_dt)

In [ ]:
#Processing the train dataset 
train_df['id_link'] = ''
train_df['preprocessed'] = ''

for i in range(0,len(train_df)):
    train_df['id_link'][i]='/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/data1/' +train_df['id'][i]+'.xml'
    # Calling function to read XML files
    preprocessed = read_xml(train_df['id_link'][i])
    train_df['preprocessed'][i]=preprocessed

In [ ]:
train_df.head()


,id,gender,id_link,preprocessed
0,d7d392835f50664fc079f0f388e147a0,male,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,youch good things to know is that sort of stuf...
1,ee40b86368137b86f51806c9f105b34b,female,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,donald the menace thankscomey a return to nat...
2,919bc742d9a22d65eab1f52b11656cab,male,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,this seems super sketch too good to be true\n...
3,15b97a08d65f22d97ca685686510b6ae,female,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,just some texts with my dad about our saturday...
4,affa98421ef5c46ca7c8f246e0a134c1,female,/content/drive/MyDrive/Colab Notebooks/Ziwen /...,irrevocably love this talented human and so pr...


In [ ]:
# test_df.head()

In [ ]:
global x_train, y_train
# Converting to preprocessed content and labels for each file to list
train_text = train_df['preprocessed'].tolist() 
train_label = train_df.gender.tolist() 

#Converting the preprocessed content for each file to list ##### CHANGE HERE

# test_text = test_df['preprocessed'].tolist() 
# test_text = test_df['clean_text'].tolist()
# Creating a list for removal of stopwords and punctuations
stopwords_fin = stop_word+list(string.punctuation)

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# Defining the tokenizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl=WordNetLemmatizer()
    def __call__(self,doc):
        #using regex tokenizer,extracting the words with whitespaces alone
        output = ''.join(re.findall('[a-zA-Z\s]+',doc))
        return [self.wnl.lemmatize(t) for t in word_tokenize(output)]

# TF-IDF vectorization
vectorizer=TfidfVectorizer(analyzer='word',input='content',
                           lowercase=True,
                           token_pattern='(?u)\\b\\w\\w+\\b',
                           min_df=0.05,
                           max_df=0.95,
                           ngram_range=(1,3),
                           tokenizer=LemmaTokenizer(),
                           stop_words=stopwords_fin)


# Fitting the data and converting to arrays
x_train=vectorizer.fit_transform(train_text)
y_train=np.asarray(train_label)


# Use the same vectorizer to transform the test set
# x_test=vectorizer.transform(test_text)


# A wordcloud is generated for the for the highest frequency values of the feature names generated by the vectorizer.

#https://stackoverflow.com/questions/61916096/word-cloud-built-out-of-tf-idf-vectorizer-function
feature_names = vectorizer.get_feature_names()
dense = x_train.todense()
lst1 = dense.tolist()
#storing the details in a dataframe
df = pd.DataFrame(lst1, columns=feature_names)
df.T.sum(axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 a

able          22.108251
absolute      16.680364
absolutely    28.350321
accept        11.350078
access        14.887067
                ...    
youve         28.086061
yr            23.721974
yup           11.897810
zealand       23.815221
zero           9.346532
Length: 1862, dtype: float64

In [ ]:
hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
# hashtags = ['F1']
for hash in hashtags:
        # device = torch.device("cuda")

        test_df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/{hash}_USER_scores_with_offensiveness_irony_new.csv')
        test_text = test_df['text'].tolist()
        global x_test
        x_test=vectorizer.transform(test_text)

        # # Fitting the data and converting to arrays
        # x_train=vectorizer.fit_transform(train_text)
        # y_train=np.asarray(train_label)

        #Building a classfier for logistic Regression
        lr = LogisticRegression()
        model_name = lr.__class__.__name__
        #fit the model to the data
        lr.fit(x_train, y_train)
        print(model_name)
        # Do the prediction
        lr_predict=lr.predict(x_test)

        #Building a classifier for Linear SVC
        svc = SVC(C=0.8,gamma=1,degree=3,kernel='linear')
        model_name = svc.__class__.__name__
        #Fit the model to the data
        svc.fit(x_train, y_train)
        print(model_name)
        # Do the predictions
        svc_predict=svc.predict(x_test)

        #Building the classifier for Random Forest
        rf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=100, verbose=0, warm_start=False,class_weight=None)
        model_name = rf.__class__.__name__
        #Fit the model for the data
        rf.fit(x_train, y_train)
        print(model_name)
        # Do the predictions
        rf_predict=rf.predict(x_test)

        #https://www.geeksforgeeks.org/python-statistics-mode-function/#:~:text=Python%20statistics%20%7C%20mode%20function,most%20likely%20to%20be%20sampled.&text=Syntax%20%3A%20mode(%5Bdata%2Dset,numbers%20as%20well%20as%20Strings.
        #Finding the mode (most number of occcuring ) of the predictions into a list
        best_predictions = [mode(prediction) for prediction in zip(lr_predict,svc_predict,rf_predict)]
        print(best_predictions)
        

        # predictions = test_df.gender.to_list()
        test_df['gender'] =  best_predictions
        test_df['gender'] = test_df['gender'].map({'male': 1, 'female': 0})
        test_df.head()
        test_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/{hash}_USER_scores_with_gender_FINAL_new.csv')

LogisticRegression
SVC
RandomForestClassifier
['male', 'male', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'female', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'male', 'male', 'male', 'female', 'male', 'female',

In [ ]:
# print(type(x_test))

# Test for accuracy

In [ ]:
test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/test_labels.csv") # reading test data


In [ ]:
# Processing the test dataset
test_df['id_link'] = ''
test_df['clean_text'] = ''

for i in range(0,len(test_df)):
    test_df['id_link'][i]='/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/gender_classification/data1/' +test_df['id'][i]+'.xml'
    # Calling function to read XML files
    preprocessed = read_xml(test_df['id_link'][i])
    test_df['preprocessed'][i]=preprocessed

In [ ]:
#Converting the preprocessed content for each file to list
test_text = test_df.preprocessed.tolist() 

In [ ]:
# Use the same vectorizer to transform the test set
x_test=vectorizer.transform(test_text)

In [ ]:
x_test

<500x1862 sparse matrix of type '<class 'numpy.float64'>'
	with 140825 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics import f1_score


#Building a classfier for logistic Regression
lr = LogisticRegression()
model_name = lr.__class__.__name__
#fit the model to the data
lr.fit(x_train, y_train)
print(model_name)
# Do the prediction
lr_predict=lr.predict(x_test)


#Building a classifier for Linear SVC
svc = SVC(C=0.8,gamma=1,degree=3,kernel='linear')
model_name = svc.__class__.__name__
#Fit the model to the data
svc.fit(x_train, y_train)
print(model_name)
# Do the predictions
svc_predict=svc.predict(x_test)

#Building the classifier for Random Forest
rf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=100, verbose=0, warm_start=False,class_weight=None)
model_name = rf.__class__.__name__
#Fit the model for the data
rf.fit(x_train, y_train)
print(model_name)
# Do the predictions
rf_predict=rf.predict(x_test)

#https://www.geeksforgeeks.org/python-statistics-mode-function/#:~:text=Python%20statistics%20%7C%20mode%20function,most%20likely%20to%20be%20sampled.&text=Syntax%20%3A%20mode(%5Bdata%2Dset,numbers%20as%20well%20as%20Strings.
#Finding the mode (most number of occcuring ) of the predictions into a list
best_predictions = [mode(prediction) for prediction in zip(lr_predict,svc_predict,rf_predict)]

print(len(best_predictions))

LogisticRegression
SVC
RandomForestClassifier
500


In [ ]:
print(best_predictions)


['male', 'female', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'male', 'male', 'male', 'female', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'male', 'male', 'female', 'male', 'male', 'female', 'male', 'female', 'male', 'female', 'female', 'female', 'male', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'male', 'male', 'female', 'female', 'male', 'female', 'female', 'male', 'male', 'female', 'fe

In [ ]:
predictions

['male',
 'female',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'female',
 'female',
 'male',
 'male',
 'male',
 'male',
 'female',
 'female',
 'male',
 'male',
 'male',
 'female',
 'male',
 'male',
 'male',
 'male',
 'male',
 'female',
 'male',
 'female',
 'female',
 'male',
 'female',
 'male',
 'male',
 'male',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'male',
 'female',
 'male',
 'male',
 'female',
 'male',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'female',
 'female',
 'female',
 'female',
 'male',
 'male',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'male',
 'male',
 'female',
 'male',
 'male',
 'female',
 'male',
 'female',
 'male',
 'female',
 'female',
 'female',
 'male',
 'male',
 'female',
 'female',
 'female',
 'female',
 'female',
 'female',
 'female',
 'male',
 'female',
 'male',
 'female',
 'male',
 'f

In [ ]:
test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/test_labels.csv")
#Store the gender labels in the predictions list
predictions=test_df.gender.to_list()

from sklearn.metrics import recall_score, precision_score, f1_score,accuracy_score
print(len(predictions))
print("Accuracy_score of ensemble model:",accuracy_score(predictions,best_predictions))
print( 'F1 score:', f1_score(predictions,best_predictions, average="binary", pos_label="male"))
print( 'Recall:', recall_score(predictions,best_predictions, average="binary", pos_label="male"))
print( 'Precision:', precision_score(predictions,best_predictions, average="binary", pos_label="male"))



500
Accuracy_score of ensemble model: 0.816
F1 score: 0.8203125000000001
Recall: 0.8467741935483871
Precision: 0.7954545454545454


In [ ]:
# #Read the test data
# import torch
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/gender_classification/test_labels.csv") 
# #Store the gender labels in the predictions list
# predictions=test_df.gender.to_list()
# print("Accuracy_score of ensemble model:",accuracy_score(predictions,best_predictions)) #### Accuracy_score of ensemble model: 0.816

In [ ]:
# The gender labels which are predicted are stored in a dataframe and then exported to a csv file

#Output dataframe with the gender labels and document ids
output=pd.DataFrame(list(zip(test_df['id'].tolist(),best_predictions)),columns=['id','gender'])
#male and female are converted to 1 & 0
output['gender'] = output['gender'].map({'male': 1, 'female': 0})
output.head()

,id,gender
0,1559034163583504387,0
1,1559034163604381696,1
2,1559034163625447425,0
3,1559034244932018179,1
4,1559034330759962624,1


Use for data


# END

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python main.py 5

python3: can't open file 'main.py': [Errno 2] No such file or directory


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments

import pandas as pd

df = pd.read_csv('data/test_reddit_sub.csv')

df = df.iloc[0:3000]

FileNotFoundError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_tokencount = 128

encoding = tokenizer.batch_encode_plus(df.ReviewText.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=max_tokencount, add_special_tokens=True)

n =len(encoding['input_ids'])

encodes = [ {'input_ids':encoding['input_ids'][i], 'token_type_ids': encoding['token_type_ids'][i], 'token_type_ids': encoding['token_type_ids'][i] } for i in range(n) ]

df['gender_tokens'] = encodes

In [ ]:
df.Gender.ravel()

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          item["labels"] = float(item["labels"])
          return item

      def __len__(self):
          return len(self.labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

valid_dataset = MakeTorchData(encoding, df.Gender.ravel())

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


training_args = TrainingArguments("test-trainer")

models = ['reddit_bert','reddit_bert_base_epoch_1','reddit_bert_base_epoch_2',
          'reddit_bert_base_epoch_3','reddit_bert_base_epoch_4', 'reddit_bert_true',
          'reddit_dbert_true', 'reddit_epoch_4']

for model_ in models:
    print (f'EVAL ON MODEL {model_}: \n')

    MODEL = f'models/{model_}'

    model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2, local_files_only=True)

    trainer = Trainer(
        model,
        training_args,
        tokenizer=tokenizer,
        data_collator=data_collator,
        eval_dataset = valid_dataset,
        compute_metrics=compute_metrics,
        

    )

    print('running prediction')

    trainer.evaluate()

    print('done w this model \n\n')